<a href="https://colab.research.google.com/github/undefinedzack/stock-market-prediction-using-sentiment-analysis/blob/master/FinalDestination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Data Manipulation

import numpy as np
import pandas as pd
import re

# Preprocessing the input data

import nltk
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Creating ngrams and vectorizing the data

from gensim.models import Word2Vec, Phrases
from gensim.models.phrases import Phraser

# Tools for building a model

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

In [13]:
%cd drive/MyDrive/Colab_Data/

%ls -l

/content/drive/MyDrive/Colab_Data
total 410956
-rw------- 1 root root   2586020 Feb 20 13:52 200features_10minwords
-rw------- 1 root root 407958406 Feb 19 10:33 causeSheDidItThisWay.csv
drwx------ 4 root root      4096 Mar 19 13:58 saved_model/
-rw------- 1 root root    479968 Feb 20 13:09 stock_data.csv
-rw------- 1 root root   7076794 Feb 16 13:29 stockerbot-export1.csv
-rw------- 1 root root   1752624 Feb 16 13:31 tweet_sentiment.csv
-rw------- 1 root root    959890 Feb 20 13:28 tweets_labelled.csv


In [16]:
# creating dataframe

# df1 = pd.read_csv('stock_data.csv')
# df1 = pd.read_csv('stockerbot-export1.csv')

df1 = pd.read_csv('tweet_sentiment.csv')
# df = pd.read_csv('tweets_labelled.csv', error_bad_lines=False)

In [ ]:
df1.head(20)

,cleaned_tweets,sentiment
0,video offic mind busi david solomon tell gs in...,0
1,price lumber lb f sinc hit ytd high maci turna...,0
2,say american dream dead,-1
3,barri silbert extrem optimist bitcoin predict ...,1
4,satellit avoid attack space junk circl earth paid,-1
5,david butler favorit fang stock realmoneysod a...,0
6,miss convo one favorit thinker,0
7,u intellig document nelson mandela made public,0
8,senat want emerg alert go netflix spotifi etc,1
9,hedg fund manag marc larsi say bitcoin k possibl,0


In [17]:
text = []
sentiment = []

for i in range(len(df1)):
  text.append(df1['cleaned_tweets'][i])
  sentiment.append(df1['sentiment'][i])

# for i in range(len(df2)):
#   text.append(df2['cleaned_tweets'][i])
#   sentiment.append(df2['sentiment'][i])

df = pd.DataFrame()                     # create Data Frame
df['text'] = np.array(text)
df['sentiment'] = np.array(sentiment)

df.head(5)

,text,sentiment
0,video offic mind busi david solomon tell gs in...,0
1,price lumber lb f sinc hit ytd high maci turna...,0
2,say american dream dead,-1
3,barri silbert extrem optimist bitcoin predict ...,1
4,satellit avoid attack space junk circl earth paid,-1


In [18]:
sentiments = []
for i in range(len(df)):
  if df['sentiment'][i] == -1:
    sentiments.append(0)
  else:
    sentiments.append(1)

df['Sentiments'] = np.array(sentiments)

# Setup for Cleaning

In [19]:
non_letters = '[^A-Za-z\s]'

In [20]:
def clean(tweet :str) -> str:
  
  # removing HTML
  text = BeautifulSoup(tweet, "lxml").get_text()

  # remove non-letters
  letters_only = re.sub(non_letters, " ", text)

  # converting to lower-case
  lowercase_letters = letters_only.lower()

  return lowercase_letters

In [21]:
lemmatizer = WordNetLemmatizer()

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words("english")) 

def lemmatize(tokens :list) -> list:
  
  # lemmatize
  lemmatized_tokens = list(map(lemmatizer.lemmatize, tokens))

  # remove stop words
  # meaningful_words = list(filter(lambda x : x not in stop_words, lemmatized_tokens))

  return lemmatized_tokens


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [22]:
def preprocess(tweet :str) -> list:

  # clean tweet
  clean_tweet = clean(tweet)

  # tokenize
  tokens = word_tokenize(clean_tweet)

  # lemmatize
  lemmaz = lemmatize(tokens)

  return lemmaz

# Data Cleaning



In [ ]:
df.head()

,text,sentiment,Sentiments
0,video offic mind busi david solomon tell gs in...,0,1
1,price lumber lb f sinc hit ytd high maci turna...,0,1
2,say american dream dead,-1,0
3,barri silbert extrem optimist bitcoin predict ...,1,1
4,satellit avoid attack space junk circl earth paid,-1,0


In [23]:
tweets = np.array(df['text'])

# cleaned_data = np.array(list(map(lambda x : preprocess(x), tweets )))
cleaned_data = np.array(list(map(preprocess, tweets )))

print(cleaned_data)

[list(['video', 'offic', 'mind', 'busi', 'david', 'solomon', 'tell', 'g', 'intern', 'learn', 'wa'])
 list(['price', 'lumber', 'lb', 'f', 'sinc', 'hit', 'ytd', 'high', 'maci', 'turnaround', 'still', 'happen'])
 list(['say', 'american', 'dream', 'dead']) ...
 list(['rt', 'hd', 'nuff', 'said', 'tel', 'telcoin', 'telfam', 'crypto', 'blockchain', 'ethereum', 'bitcoin', 'btc', 'eth'])
 list(['btc'])
 list(['stellar', 'xlm', 'price', 'binanc', 'registr', 'open', 'limit', 'time'])]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


In [24]:
# bigrams

%%time
bigrams = Phrases(sentences=cleaned_data)



CPU times: user 571 ms, sys: 1.79 ms, total: 573 ms
Wall time: 572 ms


In [25]:
# trigrams

%%time
trigrams = Phrases(sentences=bigrams[cleaned_data])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 1.76 s, sys: 6.44 ms, total: 1.77 s
Wall time: 1.77 s


In [26]:
# creating trigram model

%%time
embedding_vector_size = 256
trigrams_model = Word2Vec(
    sentences = trigrams[bigrams[cleaned_data]],
    size = embedding_vector_size,
    min_count=3, window=5, workers=4)



/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 21.6 s, sys: 53.9 ms, total: 21.7 s
Wall time: 19 s


In [ ]:
print("Vocabulary size:", len(trigrams_model.wv.vocab))

Vocabulary size: 3171


In [ ]:
trigrams_model.wv.most_similar('high')

[('play', 0.999887228012085),
 ('upsid', 0.9998855590820312),
 ('day', 0.9998511075973511),
 ('action', 0.9998103380203247),
 ('aug_call', 0.9997751116752625),
 ('flow', 0.9997194409370422),
 ('credit', 0.9996976852416992),
 ('open', 0.9996654987335205),
 ('hard', 0.9995763897895813),
 ('activ', 0.9995547533035278)]

In [ ]:
trigrams_model.wv.vocab

{'video': <gensim.models.keyedvectors.Vocab at 0x7fa2649ca890>,
 'offic': <gensim.models.keyedvectors.Vocab at 0x7fa2649cb990>,
 'mind': <gensim.models.keyedvectors.Vocab at 0x7fa2649cbb10>,
 'busi': <gensim.models.keyedvectors.Vocab at 0x7fa2649cbfd0>,
 'david': <gensim.models.keyedvectors.Vocab at 0x7fa2649cca90>,
 'tell': <gensim.models.keyedvectors.Vocab at 0x7fa2649cc710>,
 'g': <gensim.models.keyedvectors.Vocab at 0x7fa2649ccd10>,
 'intern': <gensim.models.keyedvectors.Vocab at 0x7fa2649ccd50>,
 'learn': <gensim.models.keyedvectors.Vocab at 0x7fa2649cc790>,
 'wa': <gensim.models.keyedvectors.Vocab at 0x7fa2649cc390>,
 'price_lumber_lb_f': <gensim.models.keyedvectors.Vocab at 0x7fa2649cc450>,
 'sinc_hit_ytd_high': <gensim.models.keyedvectors.Vocab at 0x7fa2649ccf50>,
 'maci_turnaround_still_happen': <gensim.models.keyedvectors.Vocab at 0x7fa2649cc810>,
 'say': <gensim.models.keyedvectors.Vocab at 0x7fa2649ccbd0>,
 'american': <gensim.models.keyedvectors.Vocab at 0x7fa2649cc990>,
 

In [ ]:
keys = list(trigrams_model.wv.vocab.keys())
keys.index('slow')
  

4833

In [27]:
def vectorize_data(data, vocab: dict) -> list:
    print('Vectorize sentences...')
    keys = list(vocab.keys())
    filter_unknown = lambda word: vocab.get(word, None) is not None
    encode = lambda tweet: list(map(keys.index, filter(filter_unknown, tweet)))
    vectorized = list(map(encode, data))
    print('Vectorize sentences... (done)')
    return vectorized

print('Convert sentences to sentences with ngrams...', end='\r')
X_data = trigrams[bigrams[cleaned_data]]
print('Convert sentences to sentences with ngrams... (done)')
input_length = 150
X_pad = pad_sequences(
    sequences=vectorize_data(X_data, vocab=trigrams_model.wv.vocab),
    maxlen=input_length,
    padding='post')
print('Transform sentences to sequences... (done)')

Convert sentences to sentences with ngrams... (done)
Vectorize sentences...


/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Vectorize sentences... (done)
Transform sentences to sequences... (done)


In [ ]:
df.head(20)

,text,sentiment,Sentiments
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1,1
2,user I'd be afraid to short AMZN - they are lo...,1,1
3,MNTA Over 12.00,1,1
4,OI Over 21.37,1,1
5,PGNX Over 3.04,1,1
6,AAP - user if so then the current downtrend wi...,-1,0
7,Monday's relative weakness. NYX WIN TIE TAP IC...,-1,0
8,GOOG - ower trend line channel test & volume s...,1,1
9,AAP will watch tomorrow for ONG entry.,1,1


In [28]:
sentiments = df['Sentiments']
# sentiments = np.array(sentiments)

X_train, X_test, y_train, y_test = train_test_split(
    X_pad,
    sentiments,
    test_size=0.2,
    shuffle=True,
    random_state=42)

# Model 

In [ ]:
def build_model(embedding_matrix: np.ndarray, input_length: int):
    model = Sequential()
    model.add(Embedding(
        input_dim = embedding_matrix.shape[0],
        output_dim = embedding_matrix.shape[1], 
        input_length = input_length,
        weights = [embedding_matrix],
        trainable=True))
    model.add(Bidirectional(LSTM(128, recurrent_dropout=0.1)))
    model.add(Dropout(0.35))
    model.add(Dense(32))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    return model

model = build_model(
    embedding_matrix=trigrams_model.wv.vectors,
    input_length=input_length)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 256)          2733824   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               394240    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 3,136,321
Trainable params: 3,136,321
Non-trainable params: 0
______________________________________________

In [ ]:
model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy'])

history = model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_test, y_test),
    batch_size=100,
    epochs=5)

Epoch 1/5
228/228 [==============================] - 291s 1s/step - loss: 0.3164 - accuracy: 0.9058 - val_loss: 0.1093 - val_accuracy: 0.9590
Epoch 2/5
228/228 [==============================] - 282s 1s/step - loss: 0.0542 - accuracy: 0.9834 - val_loss: 0.0859 - val_accuracy: 0.9706
Epoch 3/5
228/228 [==============================] - 275s 1s/step - loss: 0.0169 - accuracy: 0.9944 - val_loss: 0.1161 - val_accuracy: 0.9724
Epoch 4/5
228/228 [==============================] - 271s 1s/step - loss: 0.0080 - accuracy: 0.9979 - val_loss: 0.1151 - val_accuracy: 0.9740
Epoch 5/5
228/228 [==============================] - 278s 1s/step - loss: 0.0054 - accuracy: 0.9986 - val_loss: 0.1478 - val_accuracy: 0.9742


In [ ]:
!pwd

/content/drive/My Drive/Colab_Data


In [ ]:
model.save('saved_model')

INFO:tensorflow:Assets written to: saved_model/assets


In [14]:
!ls

200features_10minwords	  stock_data.csv	  tweets_labelled.csv
causeSheDidItThisWay.csv  stockerbot-export1.csv
saved_model		  tweet_sentiment.csv


In [15]:
new_model = tf.keras.models.load_model('saved_model')

# architecture
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 256)          2733824   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               394240    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 3,136,321
Trainable params: 3,136,321
Non-trainable params: 0
______________________________________________

In [32]:
predictions = new_model.predict(X_pad)

In [33]:
predictions

array([[0.9951609 ],
       [0.9999325 ],
       [0.00384038],
       ...,
       [0.9999994 ],
       [0.99949574],
       [0.9999999 ]], dtype=float32)

In [34]:
predictions = np.round(predictions)
print(predictions)

[[1.]
 [1.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]


In [35]:
sentiments

0        1
1        1
2        0
3        1
4        0
        ..
28435    1
28436    1
28437    1
28438    1
28439    1
Name: Sentiments, Length: 28440, dtype: int64

In [37]:
knt = 0
for i in range(len(predictions)):
  if predictions[i][0] == sentiments[i]/1:
    knt += 1
    print(f'{predictions[i][0]} : {sentiments[i]/1}')

print(knt, print(len(sentiments)))

Streaming output truncated to the last 5000 lines.
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
0.0 : 0.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
0.0 : 0.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
0.0 : 0.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
0.0 : 0.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
0.0 : 0.0
0.0 : 0.0
0.0 : 0.0
1.0 : 1.0
0.0 : 0.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
0.0 : 0.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
0.0 : 0.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
0.0 : 0.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0
1.0 : 1.0